In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

from computeDistance import *

In [ ]:
# Pick the needed file
csv_filename = 'video/stage-large_cam_1_30fps_635-665out.csv'

# get the raw boxes
raw_boxes_output, box_per_frame = get_raw_boxes_information(csv_filename)

# perform non max suppression
boxes_output = non_max_suppression(raw_boxes_output, iou_threshold = 0.7)

In [ ]:
# Display the mean human box size per frame (with and without non max suppression)
detect_threshold = 0.
raw_mean_human_size = get_mean_human_boxes_size_per_frame(raw_boxes_output, detect_threshold = detect_threshold)
mean_human_size = get_mean_human_boxes_size_per_frame(boxes_output, detect_threshold = detect_threshold)
plt.plot(np.arange(len(mean_human_size)), raw_mean_human_size, label = 'raw')
plt.plot(np.arange(len(mean_human_size)), mean_human_size, label='non max suppression')
plt.xlabel('Frame')
plt.ylabel('Mean human box size')
plt.legend()

plt.show()

In [ ]:
# Display the histogram of the human box size
raw_human_box_hist = get_human_boxes_size_hist(raw_boxes_output, detect_threshold = detect_threshold)
human_box_hist = get_human_boxes_size_hist(boxes_output, detect_threshold = detect_threshold)

plt.hist(raw_human_box_hist, 50, normed=1, facecolor='blue', alpha=0.5, label='raw')
plt.hist(human_box_hist, 50, normed=1, facecolor='orange', alpha=0.5, label='non max suppression')
plt.xlabel('Human box size')
plt.ylabel('# occurrences')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Display the histogram of the detection scores
raw_scores_hist = get_detection_score_hist(raw_boxes_output)
score_hist = get_detection_score_hist(boxes_output)

plt.hist(raw_scores_hist, 50, normed=1, facecolor='blue', alpha=0.5, label='raw')
plt.hist(score_hist, 50, normed=1, facecolor='orange', alpha=0.5, label='non max suppression')
plt.xlabel('detection score')
plt.ylabel('# occurrences')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Display the number of detected human per frame (with and without non max suppression)
detect_threshold = 0.65
size_threshold = 0.02
raw_humans_per_frame = get_human_boxes_per_frame(raw_boxes_output, 
                                                 detect_threshold = detect_threshold, 
                                                 size_threshold = size_threshold)
humans_per_frame = get_human_boxes_per_frame(boxes_output, 
                                             detect_threshold = detect_threshold, 
                                             size_threshold = size_threshold)

print('non max suppression mean humans per frame: ', np.mean(humans_per_frame))

plt.plot(np.arange(len(humans_per_frame)), raw_humans_per_frame,  label='raw')
plt.plot(np.arange(len(humans_per_frame)), humans_per_frame, label='non max suppression')
plt.xlabel('Frame')
plt.ylabel('Number of human boxes')
plt.legend()
plt.show()

In [ ]:
# Perform curve smoothing on the humans per frame data
width = 31
poly_order = 1
savgol_humans_per_frame = savgol_filter(humans_per_frame, width, poly_order)
handmade_smoothing = curve_smoothing(humans_per_frame, width = width)

plt.plot(np.arange(len(savgol_humans_per_frame)), savgol_humans_per_frame, label='savgol filter')
plt.plot(np.arange(len(savgol_humans_per_frame)), handmade_smoothing, label='handmade filter')

plt.xlabel('Frame')
plt.ylabel('Number of human boxes')
plt.legend()
plt.show()

In [ ]:
# Perform the all chain of distance computation
# Give all the parameters
duration = 20
size_threshold = 0.02
mean_size_threshold = 0.02
detect_threshold = 0.65
iou_threshold=0.7
dist = compute_frame_classification(csv_filename = csv_filename, video_duration = duration, 
                                    size_threshold = size_threshold, mean_size_threshold = mean_size_threshold,
                                    detect_threshold = detect_threshold, iou_threshold = iou_threshold)

plt.plot(np.arange(len(dist)), dist)
plt.xlabel('Time (s)')
plt.ylabel('Distance to normal')
plt.show()